In [1]:

import pandas as pd
import requests
import re
from io import StringIO

url = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/PD04639PD-PD04640PD-PD04637PD-PD04638PD/csv/2020-01-01/2021-11-30"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
req = requests.get(url, headers=headers)
req_text=str(req.text).replace("<br>","\n")
req_text=str(req_text).replace('"',"")

dict_months = {
    "Ene": "01",
    "Feb": "02",
    "Mar": "03",
    "Abr": "04",
    "May": "05",
    "Jun": "06",
    "Jul": "07",
    "Ago": "08",
    "Set": "09",
    "Oct": "10",
    "Nov": "11",
    "Dic": "12"
}

pattern = re.compile(r'\b(' + '|'.join(dict_months.keys()) + r')\b')
result = pattern.sub(lambda x: dict_months[x.group()], req_text)
data = StringIO(result)

df=pd.read_csv(data, names=["FECHA","TC_SBS_COMPRA","TC_SBS_VENTA","TC_INTERBANCARIO_COMPRA","TC_INTERBANCARIO_VENTA"],skiprows=1)
df['FECHA']=pd.to_datetime(df['FECHA'], format='%d.%m.%y')
df



,FECHA,TC_SBS_COMPRA,TC_SBS_VENTA,TC_INTERBANCARIO_COMPRA,TC_INTERBANCARIO_VENTA
0,2020-01-02,3.301000,3.303143,3.301,3.305
1,2020-01-03,3.310571,3.312000,3.308,3.311
2,2020-01-06,3.322000,3.324000,3.320,3.323
3,2020-01-07,3.325571,3.326571,3.325,3.327
4,2020-01-08,3.317571,3.319143,3.318,3.320
...,...,...,...,...,...
478,2021-11-24,4.019500,4.022000,4.018,4.023
479,2021-11-25,4.026167,4.029167,4.020,4.031
480,2021-11-26,4.037667,4.039667,4.034,4.041
481,2021-11-29,4.050500,4.054167,4.048,4.059


In [5]:

def round_half_up(number,decimals):
    if round((number * 10 ** (decimals + 1)) % 10 )== 5:
        new_number=number+1/(10**(decimals+1))
        return round(new_number, decimals)
    else:
        return round(number, decimals)

df['TC_SBS_COMPRA']=df['TC_SBS_COMPRA'].apply(round_half_up,decimals=3)
df['TC_SBS_VENTA']=df['TC_SBS_VENTA'].apply(round_half_up,decimals=3)

df['AÑO'] = df['FECHA'].dt.year

df.to_csv("../results/dolar.csv",index=False)
df


,FECHA,TC_SBS_COMPRA,TC_SBS_VENTA,TC_INTERBANCARIO_COMPRA,TC_INTERBANCARIO_VENTA,AÑO
0,2020-01-02,3.301,3.303,3.301,3.305,2020
1,2020-01-03,3.311,3.312,3.308,3.311,2020
2,2020-01-06,3.322,3.324,3.320,3.323,2020
3,2020-01-07,3.326,3.327,3.325,3.327,2020
4,2020-01-08,3.318,3.319,3.318,3.320,2020
...,...,...,...,...,...,...
478,2021-11-24,4.020,4.022,4.018,4.023,2021
479,2021-11-25,4.026,4.029,4.020,4.031,2021
480,2021-11-26,4.038,4.040,4.034,4.041,2021
481,2021-11-29,4.051,4.054,4.048,4.059,2021


In [3]:
df_sbs_com=df[["FECHA","TC_SBS_COMPRA"]]
df_sbs_ven=df[["FECHA","TC_SBS_VENTA"]]
df_int_com=df[["FECHA","TC_INTERBANCARIO_COMPRA"]]
df_int_ven=df[["FECHA","TC_INTERBANCARIO_VENTA"]]

df_sbs_com.columns=['time','value']
df_sbs_ven.columns=['time','value']
df_int_com.columns=['time','value']
df_int_ven.columns=['time','value']

df_sbs_com['time']=df_sbs_com['time'].dt.strftime('%Y-%m-%d')
df_sbs_ven['time']=df_sbs_ven['time'].dt.strftime('%Y-%m-%d')
df_int_com['time']=df_int_com['time'].dt.strftime('%Y-%m-%d')
df_int_ven['time']=df_int_ven['time'].dt.strftime('%Y-%m-%d')


df_sbs_com.to_json("../results/timeseries_by_tc/TC_SBS_COMPRA.json",orient="records")
df_sbs_ven.to_json("../results/timeseries_by_tc/TC_SBS_VENTA.json",orient="records")
df_int_com.to_json("../results/timeseries_by_tc/TC_INTERBANCARIO_COMPRA.json",orient="records")
df_int_ven.to_json("../results/timeseries_by_tc/TC_INTERBANCARIO_VENTA.json",orient="records")


/var/folders/r2/03dyv5156xdb3s4mn5bd1n8r0000gn/T/ipykernel_60978/2451044916.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sbs_com['time']=df_sbs_com['time'].dt.strftime('%Y-%m-%d')
/var/folders/r2/03dyv5156xdb3s4mn5bd1n8r0000gn/T/ipykernel_60978/2451044916.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sbs_ven['time']=df_sbs_ven['time'].dt.strftime('%Y-%m-%d')
/var/folders/r2/03dyv5156xdb3s4mn5bd1n8r0000gn/T/ipykernel_60978/2451044916.py:13: SettingWithCopyWarning: 
A value is trying to b

In [4]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2023-10-16'